In [3]:
# @title 데이터준비
!mkdir my_data

!curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" > /dev/null

!curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1zib1GI8Q5wV08TgYBa2GagqNh4jyfXZz" -o my_data/wiki_20190620_small.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1323k  100 1323k    0     0   631k      0  0:00:02  0:00:02 --:--:-- 1848k


In [4]:
data = open('my_data/wiki_20190620_small.txt', 'r', encoding='utf-8')
lines = data.readlines()

In [5]:
# @title 어절 단위 토큰화

text = "이순신은 조선 중기의 무신이다."
tokenized_text = text.split(" ")
print(tokenized_text)

['이순신은', '조선', '중기의', '무신이다.']


In [6]:
# @title 패딩
# 최대 길이를 정의해두고, 최대 길이 보다 길면 제거하고, 짧으면 의미가 없는 토큰을 추가
max_seq_length = 10
tokenized_text += ["padding"] * (max_seq_length - len(tokenized_text))
print(tokenized_text)

['이순신은', '조선', '중기의', '무신이다.', 'padding', 'padding', 'padding', 'padding', 'padding', 'padding']


In [7]:
# @title Tokenizer class
class Tokenizer:
  def __init__(self):
    self.tokenizer_type_list = ['word']
    self.pad_token = '<pad>'
    self.max_seq_length = 10
    self.padding = False
  def tokenize(self, text, tokenizer_type):
    assert tokenizer_type in self.tokenizer_type_list, "None tokenizer type"
    if tokenizer_type == 'word':
      tokenized_text = text.split(" ")
    if self.padding:
      tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
      return tokenized_text[:self.max_seq_length]
    else:
      return tokenized_text[:self.max_seq_length]
  def batch_tokenize(self, texts, tokenizer_type):
    for i, text in enumerate(texts):
      texts[i] = self.tokenize(text, tokenizer_type)
    return texts


In [8]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = '[pad]'
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [9]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'word'))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다",'그는 임진왜란을 승리로 이끌었다'],'word'))

['이순신은', '조선', '중기의', '무신이다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']
[['이순신은', '조선', '중기의', '무신이다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]'], ['그는', '임진왜란을', '승리로', '이끌었다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']]


In [10]:
# @title 형태소 단위 토큰화

!sudo apt-get install build-essential
!sudo apt-get install python3-dev
!sudo apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install --upgrade pip setuptools wheel

!pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libmecab2 mecab-ipadic mecab-utils
The following NEW packages will be installed:
  libmecab-dev libmecab2 mecab mecab-ipadic mecab-ipadic-utf8 mecab-utils
0 upgraded, 6 newly installed, 0 to remove and 29 not upgraded.
Need to get 7,367 kB of archives.
After this operation, 59.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 97.6 MB/s eta 0:00:00
mecab-ko is already installed
Install mecab-ko-dic
Install mecab-ko-dic
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 47.4M  100 47.4M    0     0  22.6M      0  0:00:02  0:00:02 --:--:-- 42.3M
mecab-ko-dic-2.1.1-20180720/
mecab-ko-dic-2.1.1-20180720/configure
mecab-ko-dic-2.1.1-20180720/COPYING
mecab-ko-dic-2.1.1-20180720/autogen.sh
mecab-ko-dic-2.1.1-20180720/Place-station.csv
mecab-ko-dic-2.1.1-20180720/NNG.csv
mecab-ko-dic-2.1.1-20180720/README
mecab-ko-dic-2.1.1-20180720/EF.csv
mecab-ko-dic-2.1.1-20180720/MAG.csv
mecab-ko-dic-2.1.1-20180720/Preanalysis.csv
mecab-ko-dic-2.1.1-20180720/NNB.csv
mecab-ko-dic-2.1.1-20180720/Person-actor.csv
mecab-ko-dic-2.1.1-20180720/VV.csv
mecab-ko-dic-2.1.1-20180720/Makefile.in
mec

In [11]:
from konlpy.tag import Mecab

mecab = Mecab(dicpath="/usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ko-dic")

print(mecab.pos("아버지가방에들어가신다"))

[('아버지', 'NNG'), ('가', 'JKS'), ('방', 'NNG'), ('에', 'JKB'), ('들어가', 'VV'), ('신다', 'EP+EC')]


In [12]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
print(tokenized_text)

['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.']


In [13]:
# @title 형태소 Tokenizer class
class Tokenizer:
  def __init__(self):
    self.tokenizer_type_list = ['word','morph']
    self.pad_token = '<pad>'
    self.max_seq_length = 10
    self.padding = False
  def tokenize(self, text, tokenizer_type):
    assert tokenizer_type in self.tokenizer_type_list, "None tokenizer type"
    if tokenizer_type == 'word':
      tokenized_text = text.split(" ")
    elif tokenizer_type =='morph':
      tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
    if self.padding:
      tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
      return tokenized_text[:self.max_seq_length]
    else:
      return tokenized_text[:self.max_seq_length]
  def batch_tokenize(self,texts, tokenizer_type):
    for i, text in enumerate(texts):
        texts[i] = self.tokenize(text, tokenizer_type)
    return texts


In [14]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = '[pad]'
my_tokenizer.max_seq_length = 10
my_tokenizer.padding = True

In [15]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'morph'))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."],'morph'))

['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '[pad]', '[pad]']
[['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '.', '[pad]'], ['그', '는', '임진왜란', '을', '승리', '로', '이끌', '었', '다', '.']]


In [16]:
# @title 음절 단위 토큰화

In [17]:
text = "이순신은 조선 중기의 무신이다."
tokenized_text = list((text))
print(tokenized_text)

['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '.']


In [18]:
# @title 자소단위 Tokenizer class
class Tokenizer:
  def __init__(self):
    self.tokenizer_type_list = ['word','morph','syllable']
    self.pad_token = '<pad>'
    self.max_seq_length = 10
    self.padding = False
  def tokenize(self, text, tokenizer_type):
    assert tokenizer_type in self.tokenizer_type_list, "None tokenizer type"
    if tokenizer_type == 'word':
      tokenized_text = text.split(" ")
    elif tokenizer_type =='morph':
      tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
    elif tokenizer_type == 'syllable':
      tokenized_text = list(text)
    if self.padding:
      tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
      return tokenized_text[:self.max_seq_length]
    else:
      return tokenized_text[:self.max_seq_length]
  def batch_tokenize(self,texts, tokenizer_type):
    for i, text in enumerate(texts):
        texts[i] = self.tokenize(text, tokenizer_type)
    return texts


In [19]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = '[pad]'
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [20]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'syllable'))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."],'syllable'))

['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '[pad]', '[pad]', '[pad]', '[pad]']
[['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '.', '[pad]', '[pad]', '[pad]'], ['그', '는', ' ', '임', '진', '왜', '란', '을', ' ', '승', '리', '로', ' ', '이', '끌', '었', '다', '.', '[pad]', '[pad]']]


In [21]:
# @title 자소 단위 토큰화
# 최대 초성, 중성, 종성 3개의 자소로 분리가 가능

!pip install hgtk

In [22]:
import hgtk

text = "이순신은 조선 중기의 무신이다."
tokenized_text = list(hgtk.text.decompose(text))
print(tokenized_text)

['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ', 'ㅓ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅜ', 'ㅇ', 'ᴥ', 'ㄱ', 'ㅣ', 'ᴥ', 'ㅇ', 'ㅢ', 'ᴥ', ' ', 'ㅁ', 'ㅜ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅣ', 'ᴥ', 'ㄷ', 'ㅏ', 'ᴥ', '.']


In [23]:
# @title 자소단위 Tokenizer class
class Tokenizer:
  def __init__(self):
    self.tokenizer_type_list = ['word','morph','syllable','jaso']
    self.pad_token = '<pad>'
    self.max_seq_length = 10
    self.padding = False
  def tokenize(self, text, tokenizer_type):
    assert tokenizer_type in self.tokenizer_type_list, "None tokenizer type"
    if tokenizer_type == 'word':
      tokenized_text = text.split(" ")
    elif tokenizer_type =='morph':
      tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
    elif tokenizer_type == 'syllable':
      tokenized_text = list(text)
    elif tokenizer_type == 'jaso':
      tokenized_text = list(hgtk.text.decompose(text))
    if self.padding:
      tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
      return tokenized_text[:self.max_seq_length]
    else:
      return tokenized_text[:self.max_seq_length]
  def batch_tokenize(self,texts, tokenizer_type):
    for i, text in enumerate(texts):
        texts[i] = self.tokenize(text, tokenizer_type)
    return texts


In [24]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = '[pad]'
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [25]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'jaso'))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."],'jaso'))

['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ']
[['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ'], ['ㄱ', 'ㅡ', 'ᴥ', 'ㄴ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅇ', 'ㅣ', 'ㅁ', 'ᴥ', 'ㅈ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅙ', 'ᴥ', 'ㄹ']]


In [26]:
# @title Wordpiece 단위 토큰화


!pip install transformers

In [27]:
!mkdir wordPeiceTokenizer

In [39]:
from tokenizers import BertWordPieceTokenizer

wp_tokenizer = BertWordPieceTokenizer(
      clean_text= True,
      handle_chinese_chars= True,
      strip_accents= False,
      lowercase= False,
)
# clean_text를 True로 해주면, 토큰화시 띄워쓰기를 자동으로 제거
# handle_chinese_chars를 True로 해주면, 한자를 음절 단위로 처리
# strip_accents는 accent에 따라 자동으로 분리를 해주는 기능
# lowercase는 모든 알파벳을 소문자로 바꿔주는 기능

In [49]:
wp_tokenizer.train(
      files="my_data/wiki_20190620_small.txt",
      vocab_size=10000,
      min_frequency=2,
      show_progress=True,
      special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
      wordpieces_prefix="##"
)

# vocab_size는 단어 사전의 크기. 단어 사전이 너무 크면 모두 음절 단위로 분리될 수 도 있음
# min_frequency는 정해진 개수 이하로 등장한 것은 사전에 추가하지 않을 수 있게 해줌
# show_progress는 처리 과정을 진행바로
# special_tokens는 앞서 배운 패딩과 같이 특수한 역할을 하는 토큰들을 의미
# wordpieces_prefix은 나눠진 글자가 하나라는 표시. (이순신 => 이, ##순신)

print(wp_tokenizer.get_vocab_size())

10000


In [50]:
import os

save_dir = "wordPieceTokenizer"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)  # 디렉터리 생성

wp_tokenizer.save_model(save_dir, "my_tokenizer")


['wordPieceTokenizer/my_tokenizer-vocab.txt']

In [51]:
text = "이순신은 조선 중기의 무신이다."

tokenized_text = wp_tokenizer.encode(text)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.']
[707, 1273, 7622, 2003, 755, 2607, 453, 8506, 1018, 16]


In [52]:
# @title wordpiece Tokenizer class
class Tokenizer:
  def __init__(self):
    self.tokenizer_type_list = ['word','morph','syllable','jaso','wordpiece']
    self.pad_token = '<pad>'
    self.max_seq_length = 10
    self.padding = False
  def tokenize(self, text, tokenizer_type):
    assert tokenizer_type in self.tokenizer_type_list, "None tokenizer type"
    if tokenizer_type == 'word':
      tokenized_text = text.split(" ")
    elif tokenizer_type =='morph':
      tokenized_text = [lemma[0] for lemma in mecab.pos(text)]
    elif tokenizer_type == 'syllable':
      tokenized_text = list(text)
    elif tokenizer_type == 'jaso':
      tokenized_text = list(hgtk.text.decompose(text))
    elif tokenizer_type == 'wordpiece':
      tokenized_text = wp_tokenizer.encode(text).tokens
    if self.padding:
      tokenized_text += [self.pad_token] * (self.max_seq_length - len(tokenized_text))
      return tokenized_text[:self.max_seq_length]
    else:
      return tokenized_text[:self.max_seq_length]
  def batch_tokenize(self,texts, tokenizer_type):
    for i, text in enumerate(texts):
        texts[i] = self.tokenize(text, tokenizer_type)
    return texts


In [53]:
my_tokenizer = Tokenizer()
my_tokenizer.pad_token = '[pad]'
my_tokenizer.max_seq_length = 20
my_tokenizer.padding = True

In [54]:
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'wordpiece'))
print(my_tokenizer.batch_tokenize(["이순신은 조선 중기의 무신이다.", "그는 임진왜란을 승리로 이끌었다."],'wordpiece'))

['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']
[['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '.', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]'], ['그는', '임진', '##왜', '##란을', '승리', '##로', '이끌었다', '.', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']]


In [55]:
# @title 구성된 함수 모두 확인
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'word'))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'morph'))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'syllable'))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'jaso'))
print(my_tokenizer.tokenize("이순신은 조선 중기의 무신이다",'wordpiece'))

['이순신은', '조선', '중기의', '무신이다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']
['이순신', '은', '조선', '중기', '의', '무신', '이', '다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']
['이', '순', '신', '은', ' ', '조', '선', ' ', '중', '기', '의', ' ', '무', '신', '이', '다', '[pad]', '[pad]', '[pad]', '[pad]']
['ㅇ', 'ㅣ', 'ᴥ', 'ㅅ', 'ㅜ', 'ㄴ', 'ᴥ', 'ㅅ', 'ㅣ', 'ㄴ', 'ᴥ', 'ㅇ', 'ㅡ', 'ㄴ', 'ᴥ', ' ', 'ㅈ', 'ㅗ', 'ᴥ', 'ㅅ']
['이', '##순', '##신은', '조선', '중', '##기의', '무', '##신이', '##다', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]', '[pad]']


# BERT 학습

In [56]:
import torch
torch.cuda.is_available()

True

In [62]:
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast

In [63]:
tokenizer = BertTokenizerFast(
    vocab_file = '/content/wordPieceTokenizer/my_tokenizer-vocab.txt',
    max_len = 128,
    do_lower_case=False,
)

In [66]:
tokenizer.add_special_tokens({'mask_token':'[MASK]'})
print(tokenizer.tokenize('이순신은 [MASK] 중기의 무신이다.'))

['이', '##순', '##신은', '[MASK]', '중', '##기의', '무', '##신이', '##다', '.']


In [68]:
config = BertConfig( # 영어기준
      vocab_size=20000,
      # hidden_size=512, # hidden vector size
      # num_hidden_layers=12, # layer num
      # num_attention_heads=8, # transformer attention head number
      # intermediate_size=3072, # transformer 내에 있는 feed-forward network의 dimension size
      # hidden_act="gelu",
      # hidden_dropout_prob=0.1,
      # attention_probs_dropout_prob=0.1,
      max_position_embeddings=128, # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정
      # type_vocab_size=2, # token type ids의 범위 (BERT는 segmentA, segmentB로 2종류)
      # pad_token_id=0,
      # position_embedding_type="absolute"
)
model = BertForPreTraining(config=config)
model.num_parameters()

101720098

In [71]:
from transformers import DataCollatorForLanguageModeling

In [79]:
from transformers.data.datasets.language_modeling import TextDatasetForNextSentencePrediction

class TextDatasetForNextSentencePrediction(Dataset):

In [72]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [73]:
from transformers import Trainer, TrainingArguments

In [75]:
training_args = TrainingArguments(
      output_dir='model_output',
      overwrite_output_dir=True,
      num_train_epochs=10,
      per_gpu_train_batch_size=32,
      save_steps=1000,
      save_total_limit=2,
      logging_steps=100
)

# output_dir은 학습도중 모델이 저장될 위치
# overwrite_output_dir는 기존의 위치를 덮어써도 되는지를 설정
# num_train_epochs는 몇번 학습할지를 설정
# per_gpu_train_batch_size는 한번에 몇개의 데이터를 학습할지 결정

In [ ]:
trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=dataset
)
trainer.train()

In [ ]:
from transformers import BertForMaskedLM, pipeline

my_model = BertForMaskedLM.from_pretrained('model_output')
nlp_fill = pipeline('fill-mask', top_k=5, model=my_model, tokenizer=tokenizer)
nlp_fill('이순신은 [MASK] 중기의 무신이다.')